<a href="https://colab.research.google.com/github/tjHendrixx/DS-Unit-1-Sprint-2-Data-Wrangling/blob/master/DS_Unit_1_Sprint_Challenge_2_Data_Wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Unit 1 Sprint Challenge 2

## Data Wrangling

In this Sprint Challenge you will use data from [Gapminder](https://www.gapminder.org/about-gapminder/), a Swedish non-profit co-founded by Hans Rosling. "Gapminder produces free teaching resources making the world understandable based on reliable statistics."
- [Cell phones (total), by country and year](https://raw.githubusercontent.com/open-numbers/ddf--gapminder--systema_globalis/master/ddf--datapoints--cell_phones_total--by--geo--time.csv)
- [Population (total), by country and year](https://raw.githubusercontent.com/open-numbers/ddf--gapminder--systema_globalis/master/ddf--datapoints--population_total--by--geo--time.csv)
- [Geo country codes](https://github.com/open-numbers/ddf--gapminder--systema_globalis/blob/master/ddf--entities--geo--country.csv)

These two links have everything you need to successfully complete the Sprint Challenge!
- [Pandas documentation: Working with Text Data](https://pandas.pydata.org/pandas-docs/stable/text.html]) (one question)
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf) (everything else)

## Part 0. Load data

You don't need to add or change anything here. Just run this cell and it loads the data for you, into three dataframes.

In [0]:
import pandas as pd

cell_phones = pd.read_csv('https://raw.githubusercontent.com/open-numbers/ddf--gapminder--systema_globalis/master/ddf--datapoints--cell_phones_total--by--geo--time.csv')

population = pd.read_csv('https://raw.githubusercontent.com/open-numbers/ddf--gapminder--systema_globalis/master/ddf--datapoints--population_total--by--geo--time.csv')

geo_country_codes = (pd.read_csv('https://raw.githubusercontent.com/open-numbers/ddf--gapminder--systema_globalis/master/ddf--entities--geo--country.csv')
                       .rename(columns={'country': 'geo', 'name': 'country'}))

## Part 1. Join data

First, join the `cell_phones` and `population` dataframes (with an inner join on `geo` and `time`).

The resulting dataframe's shape should be: (8590, 4)

In [4]:
cell_phones.head(25)



,geo,time,cell_phones_total
0,abw,1960,0.0
1,abw,1965,0.0
2,abw,1970,0.0
3,abw,1975,0.0
4,abw,1976,0.0
5,abw,1977,0.0
6,abw,1978,0.0
7,abw,1979,0.0
8,abw,1980,0.0
9,abw,1981,0.0


In [5]:
population.head()

,geo,time,population_total
0,afg,1800,3280000
1,afg,1801,3280000
2,afg,1802,3280000
3,afg,1803,3280000
4,afg,1804,3280000


In [0]:
cell_pop = pd.merge(cell_phones, population, on=['geo', 'time'])

In [7]:
cell_pop


,geo,time,cell_phones_total,population_total
0,afg,1960,0.0,8996351
1,afg,1965,0.0,9938414
2,afg,1970,0.0,11126123
3,afg,1975,0.0,12590286
4,afg,1976,0.0,12840299
5,afg,1977,0.0,13067538
6,afg,1978,0.0,13237734
7,afg,1979,0.0,13306695
8,afg,1980,0.0,13248370
9,afg,1981,0.0,13053954


In [8]:
cell_pop.shape

(8590, 4)

Then, select the `geo` and `country` columns from the `geo_country_codes` dataframe, and join with your population and cell phone data.

The resulting dataframe's shape should be: (8590, 5)

In [9]:
geo_country_codes.head()


,geo,alt_5,alternative_1,alternative_2,alternative_3,alternative_4_cdiac,arb1,arb2,arb3,arb4,...,latitude,longitude,main_religion_2008,country,pandg,un_state,unicode_region_subtag,upper_case_name,world_4region,world_6region
0,abkh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Abkhazia,NaN,False,NaN,NaN,europe,europe_central_asia
1,abw,NaN,NaN,NaN,NaN,Aruba,NaN,NaN,NaN,NaN,...,12.50000,-69.96667,christian,Aruba,NaN,False,AW,ARUBA,americas,america
2,afg,NaN,Islamic Republic of Afghanistan,NaN,NaN,Afghanistan,NaN,NaN,NaN,NaN,...,33.00000,66.00000,muslim,Afghanistan,AFGHANISTAN,True,AF,AFGHANISTAN,asia,south_asia
3,ago,NaN,NaN,NaN,NaN,Angola,NaN,NaN,NaN,NaN,...,-12.50000,18.50000,christian,Angola,ANGOLA,True,AO,ANGOLA,africa,sub_saharan_africa
4,aia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,18.21667,-63.05000,christian,Anguilla,NaN,False,AI,ANGUILLA,americas,america


In [10]:
merged = pd.merge(cell_pop, geo_country_codes, how='left, left_on')

KeyError: ignored

In [0]:
merged = cell_pop.merge(geo_country_codes[['geo', 'country']], on='geo', how='outer')

In [12]:
merged

,geo,time,cell_phones_total,population_total,country
0,afg,1960.0,0.0,8996351.0,Afghanistan
1,afg,1965.0,0.0,9938414.0,Afghanistan
2,afg,1970.0,0.0,11126123.0,Afghanistan
3,afg,1975.0,0.0,12590286.0,Afghanistan
4,afg,1976.0,0.0,12840299.0,Afghanistan
5,afg,1977.0,0.0,13067538.0,Afghanistan
6,afg,1978.0,0.0,13237734.0,Afghanistan
7,afg,1979.0,0.0,13306695.0,Afghanistan
8,afg,1980.0,0.0,13248370.0,Afghanistan
9,afg,1981.0,0.0,13053954.0,Afghanistan


In [13]:
merged.shape


(8668, 5)

## Part 2. Make features

Calculate the number of cell phones per person, and add this column onto your dataframe.

(You've calculated correctly if you get 1.220 cell phones per person in the United States in 2017.)

In [0]:
merged['Phones Per Person'] = merged['population_total']//merged['cell_phones_total']

In [15]:
merged.head(100)


,geo,time,cell_phones_total,population_total,country,Phones Per Person
0,afg,1960.0,0.0,8996351.0,Afghanistan,NaN
1,afg,1965.0,0.0,9938414.0,Afghanistan,NaN
2,afg,1970.0,0.0,11126123.0,Afghanistan,NaN
3,afg,1975.0,0.0,12590286.0,Afghanistan,NaN
4,afg,1976.0,0.0,12840299.0,Afghanistan,NaN
5,afg,1977.0,0.0,13067538.0,Afghanistan,NaN
6,afg,1978.0,0.0,13237734.0,Afghanistan,NaN
7,afg,1979.0,0.0,13306695.0,Afghanistan,NaN
8,afg,1980.0,0.0,13248370.0,Afghanistan,NaN
9,afg,1981.0,0.0,13053954.0,Afghanistan,NaN


Modify the `geo` column to make the geo codes uppercase instead of lowercase.

In [0]:
merged['geo'] = merged['geo'].str.upper()

In [0]:
merged.head(100)


## Part 3. Process data

Use the describe function, to describe your dataframe's numeric columns, and then its non-numeric columns.

(You'll see the time period ranges from 1960 to 2017, and there are 195 unique countries represented.)

In [0]:
import numpy as np

In [17]:
merged.describe(include=[np.number])

,time,cell_phones_total,population_total,Phones Per Person
count,8590.000000,8.590000e+03,8.590000e+03,4.835000e+03
mean,1994.193481,9.004950e+06,2.983823e+07,2.036213e+03
std,14.257975,5.573408e+07,1.161284e+08,2.982834e+04
min,1960.000000,0.000000e+00,4.433000e+03,0.000000e+00
25%,1983.000000,0.000000e+00,1.456148e+06,1.000000e+00
50%,1995.000000,6.200000e+03,5.725062e+06,3.000000e+00
75%,2006.000000,1.697652e+06,1.810581e+07,5.800000e+01
max,2017.000000,1.474097e+09,1.409517e+09,1.587631e+06


In [18]:
merged.describe(include=[np.object])

,geo,country
count,8668,8668
unique,273,273
top,pan,Cambodia
freq,46,46


In [19]:
merged.describe(include='all')

,geo,time,cell_phones_total,population_total,country,Phones Per Person
count,8668,8590.000000,8.590000e+03,8.590000e+03,8668,4.835000e+03
unique,273,NaN,NaN,NaN,273,NaN
top,pan,NaN,NaN,NaN,Cambodia,NaN
freq,46,NaN,NaN,NaN,46,NaN
mean,NaN,1994.193481,9.004950e+06,2.983823e+07,NaN,2.036213e+03
std,NaN,14.257975,5.573408e+07,1.161284e+08,NaN,2.982834e+04
min,NaN,1960.000000,0.000000e+00,4.433000e+03,NaN,0.000000e+00
25%,NaN,1983.000000,0.000000e+00,1.456148e+06,NaN,1.000000e+00
50%,NaN,1995.000000,6.200000e+03,5.725062e+06,NaN,3.000000e+00
75%,NaN,2006.000000,1.697652e+06,1.810581e+07,NaN,5.800000e+01


In 2017, what were the top 5 countries with the most cell phones total?

Your list of countries should have these totals:

| country | cell phones total |
|:-------:|:-----------------:|
|    ?    |     1,474,097,000 |
|    ?    |     1,168,902,277 |
|    ?    |       458,923,202 |
|    ?    |       395,881,000 |
|    ?    |       236,488,548 |



In [23]:
merged.fillna(method='mean')

ValueError: ignored

In [0]:
# This optional code formats float numbers with comma separators
pd.options.display.float_format = '{:,}'.format

2017 was the first year that China had more cell phones than people.

What was the first year that the USA had more cell phones than people?

In [0]:
merged.pivot[]

## Part 4. Reshape data

Create a pivot table:
- Columns: Years 2007—2017
- Rows: China, India, United States, Indonesia, Brazil (order doesn't matter)
- Values: Cell Phones Total

The table's shape should be: (5, 11)

#### OPTIONAL BONUS QUESTION!

Sort these 5 countries, by biggest increase in cell phones from 2007 to 2017.

Which country had 935,282,277 more cell phones in 2017 versus 2007?

If you have the time and curiosity, what other questions can you ask and answer with this data?